<a href="https://colab.research.google.com/github/taissakatrine/Transfer-Learning/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
# Carregar o dataset de gatos e cachorros
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

# Dividir o dataset em treinamento e teste
train_size = int(0.8 * len(dataset['train']))
test_size = len(dataset['train']) - train_size
train_dataset = dataset['train'].take(train_size)
test_dataset = dataset['train'].skip(train_size)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.REJR6B_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [4]:
IMG_SIZE = 224

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_dataset = train_dataset.map(format_example)
test_dataset = test_dataset.map(format_example)

BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [5]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [7]:
EPOCHS = 5
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    validation_data=test_dataset)



Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:707: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


582/582 ━━━━━━━━━━━━━━━━━━━━ 1047s 2s/step - accuracy: 0.9490 - loss: 0.1388 - val_accuracy: 0.9824 - val_loss: 0.0457
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1074s 2s/step - accuracy: 0.9852 - loss: 0.0441 - val_accuracy: 0.9843 - val_loss: 0.0420
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1092s 2s/step - accuracy: 0.9872 - loss: 0.0381 - val_accuracy: 0.9845 - val_loss: 0.0410
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1040s 2s/step - accuracy: 0.9888 - loss: 0.0344 - val_accuracy: 0.9847 - val_loss: 0.0407
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 1033s 2s/step - accuracy: 0.9897 - loss: 0.0317 - val_accuracy: 0.9845 - val_loss: 0.0408


In [8]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Accuracy: {accuracy * 100}%")


146/146 ━━━━━━━━━━━━━━━━━━━━ 214s 1s/step - accuracy: 0.9823 - loss: 0.0492
Accuracy: 98.45260977745056%


In [9]:
model.save('cats_vs_dogs_model.h5')